```
Guide to using the codebase to generate results and plots.

Usage: 
    Execute notebook or read through markdown below.

License:
    BSD 3-Clause License

    Copyright (c) 2022, James Gardner.
    All rights reserved except for those for the gwbench code which remain reserved
    by S. Borhanian; the gwbench code is included in this repository for convenience.

    Redistribution and use in source and binary forms, with or without
    modification, are permitted provided that the following conditions are met:

    1. Redistributions of source code must retain the above copyright notice, this
       list of conditions and the following disclaimer.

    2. Redistributions in binary form must reproduce the above copyright notice,
       this list of conditions and the following disclaimer in the documentation
       and/or other materials provided with the distribution.

    3. Neither the name of the copyright holder nor the names of its
       contributors may be used to endorse or promote products derived from
       this software without specific prior written permission.

    THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
    AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
    IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
    DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
    FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
    DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
    SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
    CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
    OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
    OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
```

# Guide to using codebase

This notebook explains how to use this codebase on OzStar to generate all the results and plots to compare different networks of gravitational-wave detectors.

Using %%bash magic inside Jupyter, however, does not have access to local bash aliases.

In [ ]:
%%bash
echo Hello World

In [ ]:
%%bash
lt

Instead have these directories always exist? Stop defining default paths in terms of my subsystem?

In [ ]:
%%bash
mkdir data_cosmologically_resampled_processed_injections
mkdir data_processed_injections
mkdir data_raw_injections
mkdir lambdified_functions
mkdir plots
mkdir slurm_output_files

To replicate the results and plot them:
- Execute sbatch job_scripts/job_run_all_networks_injections_at_scale.sh (job_scripts/job_run_all_networks_injections_at_scale_on_sstar.sh) on OzStar's farnarkle 1/2 (sstar) login node
    - This calls run_injections_for_network_id.py over each combinatation of network and science case (with a unique waveform per science case) for a number of primitively parallel tasks
    - This calls calculate_injections.py which initialises the network, generates the injections, and then benchmarks them by calling basic_benchmarking.py which uses gwbench. If symbolic waveforms are used, then their derivatives should first be generated and saved into lambdified_functions by executing python3 run_generate_symbolic_derivatives_for_all_locations.py. The results from each primitively parallel task are saved as .npy files in data_redshift_snr_errs_sky-area.
- Execute python3 merge_npy_files.py to combine the task files into one .npy data file for each combination of network and science case.
- During the injections, the following modules are used:
    - constants.py to define physical constants, known/measured values, and thresholds
    - filename_search_and_manipulation.py to manipulate the labels of the networks and data files
    - merger_and_detection_rates.py to define the cosmological merger rates
    - networks.py to list all studied networks
    - network_subclass.py to add commonly requested attributes to gwbench's Network class
    - useful_functions.py to define some commonly used functions, e.g. the 3-parameter sigmoid
- To then generate all of the plots into the plots directory, execute sbatch job_plot_detection_rate_and_measurement_errors.sh which calls plot_collated_detection_rate.py and plot_collated_PDFs_and_CDFs.py from the .npy data files loaded in a results class instance defined in  results_class.py.
- Asides from the above modules, the following are additionally used:
    - cosmological_redshift_resampler.py to resample the linearly uniformly sampled redshifts using a cosmological model
    - useful_plotting_functions.py to define some commonly used plotting functions
- All other files are non-critical and used for testing, e.g. profiling the memory usage of other processes